In [1]:
%classpath add mvn com.cra.figaro figaro_2.11 5.0.0.0

## Universes

Reasoning algorithms in Figaro rely on universes. These can also be "connected". Most of the time you'll use the default universe.

In [2]:
import com.cra.figaro.language._
import com.cra.figaro.language.Universe._

import com.cra.figaro.language._
import com.cra.figaro.language.Universe._


In [4]:
// e.g.
val u1 = Universe.universe
val u2 = Universe.createNew()


input is incomplete: input is incomplete

Every element corresponds to *one* universe. We can deactivate elements using *element*.deactivate(), and can re-activate it. u.active-Elements gets the active ones. 

This can be useful when we have "reference uncertainty" - we don't know the value of something that's an instance of another class (like with mutable, lazily evaluated example?)

In [10]:
import com.cra.figaro.algorithm.sampling._
import com.cra.figaro.algorithm.factored._
import com.cra.figaro.library.compound._
import com.cra.figaro.library.atomic.discrete.Uniform
import com.cra.figaro.library.atomic.continuous.Normal

import com.cra.figaro.algorithm.sampling._
import com.cra.figaro.algorithm.factored._
import com.cra.figaro.library.compound._
import com.cra.figaro.library.atomic.discrete.Uniform
import com.cra.figaro.library.atomic.continuous.Normal


### The code as listed in the tutorial won't work

For example, below:

In [9]:
abstract class Engine { val power : Element[Symbol]}
class V8 extends Engine {
    val power = Select(0.8 -> 'low, 0.2 -> 'high)
}

class V6 extends Engine {
    val power = Select(0.8 -> 'low, 0.2 -> 'high)
}

// this works fine

object MySuperEngine extends V8 {
    val power = Constant('high)
    // lol
}

<console>: 13

The error is because of how the objects share information. The working version can be found in the Figaro examples page, for this example: https://github.com/p2t2/figaro/blob/master/FigaroExamples/src/main/scala/com/cra/figaro/example/CarAndEngine.scala

You need to use "private" objects where I presume you can have more control over what's shared. This means I probably need to learn some actual Scala

In [13]:
object CarAndEngine { // the whole deal
    abstract class Engine extends ElementCollection    {
        val power : Element[Symbol]
    }

      private class V8 extends Engine {
        val power: Element[Symbol] = Select(0.8 -> 'high, 
                                            0.2 -> 'medium)("power", this) // ugh
                                  // it looks like "this" is the reference to the object,
                                  // like "self" in python
      }

      private class V6 extends Engine {
        val power: Element[Symbol] = Select(0.2 -> 'high,
                                        0.5 -> 'medium, 
                                        0.3 -> 'low)("power", this)
      }
    
    private object MySuperEngine extends V8 {
        override val power : Element[Symbol] = Constant('high)("power", this)
    }
    
    class Car extends ElementCollection {
        val engine = Uniform[Engine](new V8, new V6, MySuperEngine)("engine", this)
        val speed = CPD(
        get[Symbol]("engine.power"),
        'high -> Constant(90.0),
        'medium -> Constant(80.0),
        'low -> Constant(70.0))
    }
    
    val car = new Car
    val alg = VariableElimination(car.speed)
    alg.start()
    alg.stop()
    println(alg.expectation(car.speed)(d => d)) // what
    alg.kill()
}

85.66666666666667


$line44.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$CarAndEngine$@9c4d4ad